In [1]:
import pandas as pd

In [2]:
%ls

Data/  fake_news_model.keras  train.csv  Untitled.ipynb


In [3]:
fake_df = pd.read_csv('Data/Fake.csv')

In [4]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
true_df = pd.read_csv('Data/True.csv')

In [6]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
fake_df['label']=0
true_df['label']=1

In [8]:
df = pd.concat([fake_df,true_df],ignore_index=True)

In [9]:
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
df = df.sample(frac=1,random_state=42).reset_index(drop=True)

In [11]:
df.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1


In [51]:
df['title'][1]

'Trump drops Steve Bannon from National Security Council'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step


In [12]:
# df.to_csv('train.csv',index=False)

In [13]:
df.shape

(44898, 5)

In [14]:
print(df.isnull().sum())  # View NaN counts per column
df = df.dropna()          # Drop rows with any NaNs


title      0
text       0
subject    0
date       0
label      0
dtype: int64


In [15]:
X=df.drop('label',axis=1)
y=df['label']

In [16]:
X.shape

(44898, 4)

In [17]:
y.value_counts()

label
0    23481
1    21417
Name: count, dtype: int64

In [18]:
y.shape

(44898,)

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

2025-10-19 16:05:13.695424: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-19 16:05:14.039078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-19 16:05:15.603685: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [20]:
vocabulary_size = 5000


In [21]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [22]:
# One hot
messages = X.copy()

In [23]:
messages['title'][1]

'Trump drops Steve Bannon from National Security Council'

In [24]:
messages.reset_index(inplace=True)

In [25]:
messages.head()

,index,title,text,subject,date
0,0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017"
1,1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017"
2,2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017"
3,3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017"
4,4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016"


In [26]:
import nltk
import re
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/saugat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
text = []
for i in range(0,len(messages)):
    message = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    message = message.lower().split()
    message = [ps.stem(word) for word in message if not word in stopwords.words('english')]
    message = ' '.join(message)
    text.append(message)
    

In [29]:
text[:5]

['ben stein call th circuit court commit coup tat constitut',
 'trump drop steve bannon nation secur council',
 'puerto rico expect u lift jone act ship restrict',
 'oop trump accident confirm leak isra intellig russia video',
 'donald trump head scotland reopen golf resort']

In [30]:
one_hot_text = [one_hot(words,vocabulary_size) for words in text]
one_hot_text[:5]

[[3347, 500, 320, 4483, 4754, 2001, 3285, 4293, 2085, 3070],
 [1599, 2792, 3580, 1576, 2960, 4343, 2502],
 [3628, 1159, 3473, 949, 3030, 3208, 1257, 3305, 3953],
 [1735, 1599, 1533, 1766, 4416, 2848, 2309, 1709, 3628],
 [2007, 1599, 3340, 4721, 4331, 776, 4120]]

In [31]:
# Embedding
max_length = 20
docs = pad_sequences(one_hot_text,padding='pre',maxlen=max_length)
print(docs)

[[   0    0    0 ... 4293 2085 3070]
 [   0    0    0 ... 2960 4343 2502]
 [   0    0    0 ... 1257 3305 3953]
 ...
 [   0    0    0 ... 2919 3571 4600]
 [   0    0    0 ... 3665 2246 1582]
 [   0    0    0 ... 2193 4341 2847]]


In [32]:
# Model
embed_features = 40
model = Sequential([
    Embedding(vocabulary_size,embed_features, input_length=max_length),
    LSTM(100),
    Dense(1,activation='sigmoid')
])
model.build(input_shape=(None, max_length))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/home/saugat/miniconda3/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
E0000 00:00:1760869253.945192   52402 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1760869254.022409   52402 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [33]:
model1 = Sequential([
    Embedding(vocabulary_size,embed_features),
    Bidirectional(LSTM(100)),
    Dropout(0.3),
    Dense(1,activation='sigmoid')
])
model1.build(input_shape=(None,max_length))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
import numpy as np

In [35]:
X = np.array(docs)

In [36]:
y=np.array(y)

In [37]:
X.shape,y.shape

((44898, 20), (44898,))

In [38]:
%pip install scikit-learn
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)


Note: you may need to restart the kernel to use updated packages.


In [39]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 12s 20ms/step - accuracy: 0.9039 - loss: 0.2229 - val_accuracy: 0.9437 - val_loss: 0.1399
Epoch 2/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9846 - loss: 0.0445 - val_accuracy: 0.9352 - val_loss: 0.2240
Epoch 8/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.9871 - loss: 0.0370 - val_accuracy: 0.9367 - val_loss: 0.2502
Epoch 9/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.9902 - loss: 0.0295 - val_accuracy: 0.9347 - val_loss: 0.2644
Epoch 10/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.9928 - loss: 0.0225 - val_accuracy: 0.9344 - val_loss: 0.2865


In [40]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9001 - loss: 0.2376 - val_accuracy: 0.9400 - val_loss: 0.1466
Epoch 2/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9501 - loss: 0.1286 - val_accuracy: 0.9427 - val_loss: 0.1444
Epoch 3/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9638 - loss: 0.0961 - val_accuracy: 0.9406 - val_loss: 0.1517
Epoch 4/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9723 - loss: 0.0772 - val_accuracy: 0.9446 - val_loss: 0.1517
Epoch 5/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.9784 - loss: 0.0589 - val_accuracy: 0.9427 - val_loss: 0.1770
Epoch 6/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.9831 - loss: 0.0475 - val_accuracy: 0.9437 - val_loss: 0.1842
Epoch 7/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9872 - loss: 0.0343 - val_accuracy: 0.9414 - val_loss: 0.2277
Epoch 8/10
492/492 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.9907 - loss: 0.0270 - val_acc

In [41]:
# Performance Metrics and Accuracy

In [42]:
y_pred1=model1.predict(X_test)
y_pred= model.predict(X_test)

421/421 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
421/421 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step


In [43]:
# Prediction

In [44]:
sample_news = ["Breaking: Scientists discover cure for aging!"]


In [45]:
# Preprocess function
def preprocess_fn(text,vocabulary_size=5000, max_length=20):
    """
    Predict whether a news article is fake or real using trained model.
    """
    # Step 1: Clean text
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)

    # Step 2: Convert to sequence 
    seq = [one_hot(review, vocabulary_size)]
    
    # Step 3: Pad sequence
    padded = pad_sequences(seq, maxlen=max_length, padding='pre')
    return padded

In [46]:
def predict_news(text,model,vocabulary_size=5000,max_length=20):
    preprocessed = preprocess_fn(text,vocabulary_size,max_length)
    pred_prob = model.predict(preprocessed)[0][0]
    label = "Real"  if pred_prob > 0.5 else "fake"
    return label,pred_prob

In [47]:
label, confidence = predict_news("Breaking: Scientists discover cure for aging!", model1)
print(label, confidence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
fake 7.291063e-05


In [48]:

# Save in Keras native format
model1.save("fake_news_model.keras")
loaded_model = tf.keras.models.load_model("fake_news_model.keras")
if loaded_model:
    print("Loaded")


Loaded


In [49]:
import gradio as gr
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Load model
model = tf.keras.models.load_model('fake_news_model.keras')

# Preprocessing setup
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
vocabulary_size = 5000
max_length = 20

def preprocess_text(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    review = [ps.stem(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    seq = [one_hot(review, vocabulary_size)]
    padded = pad_sequences(seq, maxlen=max_length, padding='pre')
    return padded

def gradio_predict(text):
    preprocessed = preprocess_text(text)
    prob = model.predict(preprocessed)[0][0]
    label = 'Real' if prob > 0.5 else 'Fake'
    return f"{label} (confidence={prob:.4f})"

app = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Textbox(label="Enter News Headline or Text", lines=3, placeholder="e.g., Breaking: Scientists discover cure for aging!"),
    outputs="text",
    title="Fake News Classifier",
    description="Detect whether a news headline or article is Fake or Real using an LSTM model."
)

app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
